In [4]:
from plotly.offline import iplot
from data_loaders import TrafficAccidentDataset
import plotly.express as px
import pandas as pd
dataset = TrafficAccidentDataset(state_name = "CA", data_dir="../data/Final_Graphs",
                            #node_feature_type = args.node_feature_type,
                            use_static_edge_features=True,
                            #use_dynamic_node_features=args.load_dynamic_node_features,
                            #use_dynamic_edge_features=args.load_dynamic_edge_features,
                            #train_years=args.train_years,
                            num_negative_edges=100000000) 

monthly_data = dataset.load_monthly_data(2013, 11)

mon_data = monthly_data["temporal_node_features"].cpu().detach().numpy()
df = pd.DataFrame(mon_data)
df.columns = ["node_id","lon","lat","tavg","tmin","tmax","prcp","wspd","pres"]

print(df.head(100))
print(df.tail(100))
color_scale = [(0, 'blue'), (1,'red')]

fig = px.scatter_mapbox(df.head(1000), 
                        lat="lat", 
                        lon="lon", 
                        hover_name="node_id", 
                        hover_data=["tavg","tmin","tmax","prcp","wspd","pres"],
                        color="prcp",
                        color_continuous_scale=color_scale,
                        size="tavg",
                        zoom=8, 
                        height=800,
                        width=800)
#Need to plot node features

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#iplot(fig)#fig.write_html("test.html")

GONNA DOWNLOAD
Dataset already downloaded!


loaded static network
dict_keys(['oneway', 'length', 'bus_guideway', 'bus_stop', 'crossing', 'disused', 'elevator', 'escape', 'living_street', 'motorway', 'motorway_link', 'no', 'planned', 'primary', 'primary_link', 'razed', 'residential', 'road', 'secondary', 'secondary_link', 'street_lamp', 'street_sign', 'tertiary', 'tertiary_link', 'trail', 'trunk', 'trunk_link', 'unclassified'])


TypeError: list indices must be integers or slices, not tuple

In [25]:
from plotly.offline import iplot
from data_loaders_new import TrafficAccidentDataset
import plotly.express as px
import pandas as pd
dataset = TrafficAccidentDataset(state_name = "CA", data_dir="../data/Final_Graphs",
                            #node_feature_type = args.node_feature_type,
                            use_static_edge_features=True,
                            use_dynamic_node_features=True,
                            use_dynamic_edge_features=True,
                            #train_years=args.train_years,
                            num_negative_edges=100000000) 
monthly_data = dataset.load_monthly_data(2013, 11)

mon_data = monthly_data["temporal_node_features"].cpu().detach().numpy()
#df = pd.DataFrame(mon_data)
#df.columns = ["node_id","lon","lat","tavg","tmin","tmax","prcp","wspd","pres"]


GONNA DOWNLOAD
Dataset already downloaded!


loaded static network
dict_keys(['oneway', 'length', 'bus_guideway', 'bus_stop', 'crossing', 'disused', 'elevator', 'escape', 'living_street', 'motorway', 'motorway_link', 'no', 'planned', 'primary', 'primary_link', 'razed', 'residential', 'road', 'secondary', 'secondary_link', 'street_lamp', 'street_sign', 'tertiary', 'tertiary_link', 'trail', 'trunk', 'trunk_link', 'unclassified'])
dict_keys(['oneway', 'length', 'bus_guideway', 'bus_stop', 'crossing', 'disused', 'elevator', 'escape', 'living_street', 'motorway', 'motorway_link', 'no', 'planned', 'primary', 'primary_link', 'razed', 'residential', 'road', 'secondary', 'secondary_link', 'street_lamp', 'street_sign', 'tertiary', 'tertiary_link', 'trail', 'trunk', 'trunk_link', 'unclassified'])


In [26]:
print(len(['oneway', 'length', 'bus_guideway', 'bus_stop', 'crossing', 'disused', 'elevator', 'escape', 'living_street', 'motorway', 'motorway_link', 'no', 'planned', 'primary', 'primary_link', 'razed', 'residential', 'road', 'secondary', 'secondary_link', 'street_lamp', 'street_sign', 'tertiary', 'tertiary_link', 'trail', 'trunk', 'trunk_link', 'unclassified']))
edge_idx = monthly_data['edge_index'].cpu().detach().numpy()
edge_idx = edge_idx.T

28


In [27]:
edge_idx.shape

(3061644, 2)

In [28]:
x = monthly_data['x'].cpu().detach().numpy()

In [30]:
data = monthly_data['data']
data

Data(edge_index=[2, 3061644], edge_attr=[3061644, 26], x=[1242784, 9])

In [31]:
for i in x[0]:
    print(i)

54143556.0
-117.4331
34.516125
15.958765
10.001673
22.92505
34.50534
4.821822
1018.2


Plot the features of the road

In [36]:
import numpy as np
from numba import njit
# I want to plot the edge attributes so I need to get the index of the x from
# the edge_index


attr = monthly_data['edge_attr'].cpu().detach().numpy()


full = np.column_stack((attr,edge_idx))
df = pd.DataFrame(full)
df.columns = ['oneway', 'access_ramp', 'bus_stop', 'crossing', 'disused', 'elevator', 'escape', 'living_street', 'motorway', 'motorway_link', 'primary', 'primary_link', 'residential', 'rest_area', 'road', 'secondary', 'secondary_link', 'stairs', 'tertiary', 'tertiary_link', 'trunk', 'trunk_link', 'unclassified', 'x','unsurfaced','edge_feature','node_1_idx','node_2_idx']

def fill_lat(node_1_idx,x):
    new_column = np.empty(node_1_idx.shape[0])
    for i in range(node_1_idx.shape[0]):
        index = int(node_1_idx[i])
        new_column[i] = float(x[index, 1])  # Get the second value from the found row
    return new_column

def fill_lon(node_1_idx,x):
    new_column = np.empty(node_1_idx.shape[0])
    for i in range(node_1_idx.shape[0]):
        index = int(node_1_idx[i])
        new_column[i] = float(x[index, 2])  # Get the second value from the found row
    return new_column

node_1_values = df['node_1_idx'].to_numpy()
# Call the Numba function
lat = fill_lat(node_1_values, x)
lon = fill_lon(node_1_values, x)

# Assign the new values to a new column in the DataFrame
df['lon'] = lat
df['lat'] = lon
color_scale = [(0, 'green'), (1,'red')]
fig = px.scatter_mapbox(df.tail(1000), 
                         lat="lat", 
                         lon="lon", 
                         hover_name="node_1_idx", 
                         hover_data=['oneway', 'access_ramp', 'bus_stop', 'crossing', 'disused', 'elevator', 'escape', 'living_street', 'motorway', 'motorway_link', 'primary', 'primary_link', 'residential', 'rest_area', 'road', 'secondary', 'secondary_link', 'stairs', 'tertiary', 'tertiary_link', 'trunk', 'trunk_link', 'unclassified', 'x','unsurfaced','edge_feature','node_1_idx','node_2_idx'],
                         color="unsurfaced",
                         color_continuous_scale=color_scale,
                         zoom=8, 
                         height=550,
                         width=800)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
iplot(fig)

Plot where crashes (pos_edge) and not crashes are (neg_edge)

In [37]:

import plotly.graph_objects as go

fig = go.Figure()

safe = monthly_data['neg_edges'].cpu().detach().numpy()

df = pd.DataFrame(safe)
df.columns = ['node_1_idx','node_2_idx']

node_1_values = df['node_1_idx'].to_numpy()
# Call the Numba function
lat = fill_lat(node_1_values, x)
lon = fill_lon(node_1_values, x)

# Assign the new values to a new column in the DataFrame
df['lon'] = lat
df['lat'] = lon
color_scale = [(0, 'green'), (1,'red')]

fig.add_trace(go.Scattermapbox(
                         lat=df['lat'].head(1000), 
                         lon=df["lon"].head(1000),
                         marker=go.scattermapbox.Marker(
                            size=4,
                            color='rgb(20, 250, 20)',
                            opacity=.8
                        ),
))

not_safe = monthly_data['accidents'].cpu().detach().numpy()
how_not_safe = monthly_data['accident_counts'].cpu().detach().numpy()

df1 = pd.DataFrame(np.column_stack((not_safe,how_not_safe)))
df1.columns = ['node_1_idx','node_2_idx','accident_counts']

node_1_values = df1['node_1_idx'].to_numpy()
# Call the Numba function
lat = fill_lat(node_1_values, x)
lon = fill_lon(node_1_values, x)

# Assign the new values to a new column in the DataFrame
df1['lon'] = lat
df1['lat'] = lon
fig.add_trace(go.Scattermapbox(
                         lat=df1['lat'].head(1000), 
                         lon=df1["lon"].head(1000),
                         
                         marker=go.scattermapbox.Marker(
                            size=6,
                            color=df1['accident_counts'].head(1000),
                            opacity=1,
                            colorscale= [
                            [0, 'rgb(188,143,143)'],        #0
                            [1., 'rgb(128, 0, 0)'],             #100000

                        ],
                        ),
))


fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

iplot(fig)

Plot the distribution of safe intersections (negative edge), and positive edges

In [38]:

# Sample data
labels = ['Safe Edges', 'Not Safe Edges']
neg_height = len(safe)
pos_height = len(not_safe)
print(pos_height)
# Heights of the bars based on the lengths of the arrays
heights = [neg_height, pos_height]

# Create the bar chart
fig = go.Figure(data=[go.Bar(x=labels, y=heights)])

# Update layout
fig.update_layout(title="Comparison of this month's S/B",
                  xaxis_title='Category',
                  yaxis_title='Edge Count',
                  yaxis_type='log',
)

# Show the figure
fig.show()

356


In [1]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

Training a model

In [6]:
from . import LinkPredictor,GNN
from evaluators import Evaluator
import torch
import itertools
from trainers import *
from data_loaders import TrafficAccidentDataset
import os
import numpy as np

args = dict(batch_size=32768, device=0, dropout=0.0, encoder='gcn', epochs=1, eval_steps=1, hidden_channels=256, input_channels=128, input_time_steps=4, jk_type='last', load_dynamic_edge_features=True, load_dynamic_node_features=True, load_model_dir='none', load_model_name='epoch_50.pth', load_static_edge_features=True, log_steps=1, lr=0.01, node_feature_name='MA_ppr_128.npy', node_feature_type='verse', num_gnn_layers=2, num_negative_edges=100000000, num_predictor_layers=2, runs=1, sam_rho=0.05, sample_batch_size=10000, sample_node=False, sp_lambda=0.0001, state_name='CA', supcon_lam=0.9, supcon_tmp=0.3, test_years=[2018], train_accident_regression=True, train_sam=False, train_soft_penalty=False, train_supcon=False, train_volume_regression=False, train_years=[2016], use_time_series=False, valid_years=[2017])
args = dotdict(args)
device = f'cuda:{args.device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

dataset = TrafficAccidentDataset(state_name = 'CA', data_dir="../data/Final_Graphs",
                            node_feature_type = args.node_feature_type,
                            use_static_edge_features=args.load_static_edge_features,
                            use_dynamic_node_features=args.load_dynamic_node_features,
                            use_dynamic_edge_features=args.load_dynamic_edge_features,
                            train_years=args.train_years,
                            num_negative_edges=args.num_negative_edges) 

ImportError: cannot import name 'LinkPredictor'

In [5]:
task_type = "classification"
evaluator = Evaluator(type=task_type)

data = dataset.data
in_channels_node = data.x.shape[1] if data.x is not None else 0
in_channels_node = (in_channels_node + 6) if args.load_dynamic_node_features else in_channels_node

in_channels_edge = data.edge_attr.shape[1] if args.load_static_edge_features else 0
in_channels_edge = in_channels_edge + 1 if args.load_dynamic_edge_features else in_channels_edge

model = GNN(in_channels_node, in_channels_edge, hidden_channels=args.hidden_channels, 
                    num_layers=args.num_gnn_layers, dropout=args.dropout,
                    JK = args.jk_type, gnn_type = args.encoder, num_nodes=data.num_nodes).to(device)


feature_channels = in_channels_node if args.encoder == "none" else args.hidden_channels
if_regression = args.train_accident_regression or args.train_volume_regression
predictor = LinkPredictor(in_channels=feature_channels*2 + in_channels_edge, 
                            hidden_channels=args.hidden_channels, 
                            out_channels=1,
                            num_layers = args.num_predictor_layers,
                            dropout=args.dropout,
                            if_regression=if_regression).to(device)
params = itertools.chain(model.parameters(), predictor.parameters())
optimizer = torch.optim.Adam(params, lr=args.lr)

trainer = Trainer(model, predictor, dataset, optimizer, evaluator,
                            train_years = args.train_years,
                            valid_years = args.valid_years,
                            test_years = args.test_years,
                            epochs=args.epochs,
                            batch_size = args.batch_size,
                            eval_steps=args.eval_steps,
                            device = device,
                            log_metrics=['ROC-AUC', 'F1', 'AP', 'Recall', 'Precision'],
                            use_time_series=args.use_time_series, input_time_steps=args.input_time_steps)
            

/lustre/work/client/users/sbillingsley/.conda/envs/road-gnn-models/lib/python3.6/site-packages/torch_geometric/data/storage.py:264: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_attr', 'edge_index'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  " to suppress this warning")


In [6]:


results = {}
for run in range(args.runs):
    predictor.reset_parameters()
    params = itertools.chain(model.parameters(), predictor.parameters())
    optimizer = torch.optim.Adam(params, lr=args.lr)

    log = trainer.train(num_months=1)

    for key in log.keys():
        if key not in results:
            results[key] = []
        results[key].append(log[key])
for key in results.keys():
    print("{} : {:.2f} +/- {:.2f}".format(key, np.mean(results[key]), np.std(results[key])))


            

epoch 1, epochs left 0
year: 2016, month: 1


868
saving model
year: 2016, month: 2
920
saving model
Eval on 2016-1 data
Number of positive edges: 434 | Number of negative edges: 3060991
Eval on 2016-2 data
Number of positive edges: 460 | Number of negative edges: 3060940
Eval on 2017-1 data
Number of positive edges: 551 | Number of negative edges: 3060832
Eval on 2017-2 data
Number of positive edges: 512 | Number of negative edges: 3060853
Eval on 2018-1 data
Number of positive edges: 538 | Number of negative edges: 3060814
Eval on 2018-2 data
Number of positive edges: 541 | Number of negative edges: 3060834
MAE
Epoch: 01, Loss: 3.6539, Train: 654.37%, Valid: 774.10%, Test: 1010.29%
MSE
Epoch: 01, Loss: 3.6539, Train: 2171.50%, Valid: 3185.28%, Test: 5142.43%
---
saving model
MAE
Run 01:
Highest Train: 654.37
Valid: 774.10
Test: 1010.29
MSE
Run 01:
Highest Train: 2171.50
Valid: 3185.28
Test: 5142.43


NameError: name 'np' is not defined

Test the model

In [7]:
import os

import numpy as np
for key in results.keys():
    print("{} : {:.2f} +/- {:.2f}".format(key, np.mean(results[key]), np.std(results[key])))
load_model_dir = os.path.join("./users/sbillingsley/traffic-safety-gnn/ml_for_road_safety/epoch_1.pth")
if os.path.exists(os.path.join(load_model_dir, args.load_model_name)):
    model.load_state_dict(
        torch.load(os.path.join(load_model_dir, args.load_model_name), map_location=device)
    )
    print("Loaded model successfully!")

Train_MAE : 654.37 +/- 0.00
Valid_MAE : 774.10 +/- 0.00
Test_MAE : 1010.29 +/- 0.00
Train_MSE : 2171.50 +/- 0.00
Valid_MSE : 3185.28 +/- 0.00
Test_MSE : 5142.43 +/- 0.00


In [7]:
results, num_unsafe_edges = trainer.test_on_month_data(2018,1)

Eval on 2018-1 data
Number of positive edges: 538 | Number of negative edges: 3060814


({'MAE': tensor(0.0249), 'MSE': tensor(0.0044)}, 3061352)